<h1>Install the dependecies and create directory for the files</h1>

In [ ]:
# %pip install -q -U llama-hub unstructured
# %pip install -q -U "unstructured[doc,docx,ppt,pptx,pdf,image]"
# %pip install -q -U llama-index
# %pip install -q -U llama-index
# %pip install -q -U unstructured
# %pip install -q -U transformers
# %pip install -q -U torch
# %pip install -q -U accelerate
# %pip install -q -U llama-cpp-python
# %pip install -q -U langchain
# %pip install -q -U sentence-transformers

# Other dependencies. Already installed via terminal
# curl -L https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-MacOSX-arm64.sh -o Miniforge3-MacOSX-arm64.sh
# bash Miniforge3-MacOSX-arm64.sh
# %mkdir data

<h1>Load the data and construct the index</h1>

In [ ]:
# from pathlib import Path
import os
from llama_index import download_loader, SimpleDirectoryReader, VectorStoreIndex, ServiceContext, StorageContext, load_index_from_storage
from llama_index.llms import LlamaCPP
from llama_index.embeddings import HuggingFaceEmbedding   # use Huggingface embeddings   # loads BAAI/bge-small-en   # embed_model = HuggingFaceEmbedding()
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt
from llama_hub.file.unstructured.base import UnstructuredReader
from IPython.display import Markdown, display

UnstructuredReader = download_loader("UnstructuredReader")

def construct_index(directory_path):
    llm = LlamaCPP(
    model_path="/Users/balungelir21/Documents/LlamaIndex_Chat/codellama-13b-instruct.Q5_K_M.gguf", #Add the model Path here!!!!!!!!!!!!
    temperature=0.3,
    max_new_tokens=1024,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={},
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
    )

    embed_model_dir = "/Users/balungelir21/Documents/LlamaIndex_Chat/LlamaIndexChatbot/bge-large-en-v1.5"
    embed_model = HuggingFaceEmbedding(model_name=embed_model_dir)

    # check if storage already exists
    if(not os.path.exists("./storage")):
        # load the documents and create the index
        dir_reader = SimpleDirectoryReader(directory_path, file_extractor={
          ".txt": UnstructuredReader(),
          ".docx": UnstructuredReader(),
          ".pptx": UnstructuredReader(),
          ".pdf": UnstructuredReader(),
          ".jpg": UnstructuredReader(),
          ".png": UnstructuredReader(),
          ".eml": UnstructuredReader(),
          ".html": UnstructuredReader(),
        })
        documents = dir_reader.load_data()

        service_context = ServiceContext.from_defaults(llm=llm, chunk_size=1024, embed_model=embed_model)
        index = VectorStoreIndex.from_documents(documents, service_context=service_context)

        index.storage_context.persist()
    else:
        # load the existing index
        storage_context = StorageContext.from_defaults(persist_dir="./storage")
        index = load_index_from_storage(storage_context)

    return index

<h1>Query the index</h1>

In [ ]:
# query the index
def ask_jarvis():
    index = construct_index("./data")
    while True:
        query_engine = index.as_query_engine()
        query = input("Hello! What would you like to know?")
        response = query_engine.query(query)
        display(Markdown(f"Response: <b>{response.response}</b>"))

In [ ]:
ask_jarvis()